In [1]:

# 02_numpy_feature_engineering.ipynb

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

df = pd.read_csv("../data/customer_churn_cleaned.csv")

df.head()


,customer_id,gender,age,country,city,customer_segment,tenure_months,signup_channel,contract_type,monthly_logins,weekly_active_days,avg_session_time,features_used,usage_growth_rate,last_login_days_ago,monthly_fee,total_revenue,payment_method,payment_failures,discount_applied,price_increase_last_3m,support_tickets,avg_resolution_time,complaint_type,csat_score,escalations,email_open_rate,marketing_click_rate,nps_score,survey_response,referral_count,churn,churn_label
0,CUST_00001,Male,68,Bangladesh,London,SME,22,Web,Monthly,26,7,11.762372,5,0.06,7,30,660,PayPal,1,Yes,No,4,13.354360,Service,4.0,0,0.71,0.40,27,Satisfied,1,0,No
1,CUST_00002,Female,57,Canada,Sydney,Individual,9,Mobile,Monthly,7,5,26.846390,1,-0.28,2,30,270,Card,1,No,Yes,1,25.140088,Billing,2.0,0,0.78,0.33,-19,Neutral,2,1,Yes
2,CUST_00003,Male,24,Germany,New York,SME,58,Web,Yearly,19,5,23.380065,6,0.13,23,20,1160,Card,2,No,No,1,27.572928,Service,3.0,0,0.35,0.49,80,Neutral,1,0,No
3,CUST_00004,Male,49,Australia,Dhaka,Individual,19,Mobile,Yearly,34,7,24.243136,2,-0.17,24,30,570,Bank Transfer,0,Yes,No,3,26.420822,Technical,5.0,1,0.83,0.15,100,Neutral,0,0,No
4,CUST_00005,Male,65,Bangladesh,Delhi,Individual,52,Web,Monthly,20,6,18.872323,2,-0.16,2,50,2600,PayPal,0,No,No,0,26.674579,Technical,4.0,0,0.65,0.44,21,Unsatisfied,1,0,No


In [2]:
df['tenure_group'] = np.where(
    df['tenure_months'] < 6, 'New',
    np.where(df['tenure_months'] <= 24, 'Mid', 'Loyal')
)

df['tenure_group'].value_counts(normalize=True)


tenure_group
Loyal    0.5963
Mid      0.3178
New      0.0859
Name: proportion, dtype: float64

In [3]:
# Churn rate by tenure group
df.groupby('tenure_group')['churn'].mean()


tenure_group
Loyal    0.078316
Mid      0.086532
New      0.324796
Name: churn, dtype: float64

In [6]:
df['usage_risk_flag'] = np.where(
    (df['monthly_logins'] < 10) &
    (df['last_login_days_ago'] > 14),
    'High Risk',
    'Normal'
)

df['usage_risk_flag'].value_counts(normalize=True)


usage_risk_flag
Normal       0.9643
High Risk    0.0357
Name: proportion, dtype: float64

In [7]:
df['support_risk_flag'] = np.where(
    (df['support_tickets'] >= 3) &
    (df['csat_score'] <= 2),
    'High Risk',
    'Normal'
)

df.groupby('support_risk_flag')['churn'].mean()


support_risk_flag
High Risk    0.245370
Normal       0.098937
Name: churn, dtype: float64

In [8]:
df['combined_risk_flag'] = np.where(
    (df['usage_risk_flag'] == 'High Risk') |
    (df['support_risk_flag'] == 'High Risk'),
    'High Risk',
    'Normal'
)

df['combined_risk_flag'].value_counts(normalize=True)


combined_risk_flag
Normal       0.9434
High Risk    0.0566
Name: proportion, dtype: float64

In [9]:
df.groupby('combined_risk_flag')['churn'].mean()


combined_risk_flag
High Risk    0.199647
Normal       0.096248
Name: churn, dtype: float64

In [10]:
usage_metrics = [
    'monthly_logins',
    'weekly_active_days',
    'avg_session_time',
    'features_used',
    'usage_growth_rate',
    'last_login_days_ago'
]

df.groupby('churn')[usage_metrics].mean().T


churn,0,1
monthly_logins,19.998218,16.803134
weekly_active_days,3.486246,3.419197
avg_session_time,15.198622,15.093508
features_used,4.986747,5.026445
usage_growth_rate,0.019157,0.021763
last_login_days_ago,9.382114,10.587659
